In [2]:
from __future__ import annotations

from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import get_body
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
import glob
from astropy.stats import sigma_clip
import os
import pathlib
import pytest
from photutils.datasets import load_star_image

from astroscrappy import detect_cosmics

from astropy.table import Table
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils.aperture import ApertureStats
import matplotlib.pyplot as plt

from pathlib import Path

from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder

#uv sync, engage, ipython

ModuleNotFoundError: No module named 'pytest'

In [1]:
input_dir = Path("data")
output_file = "eagle_bias.fits"

bias_data = []

for file in sorted(input_dir.glob("Bias*.fits")):
    data = fits.getdata(file).astype('f4')

    bias_data.append(data) 
if len(bias_data) == 0:
    raise RuntimeError(f"No bias")

bias_3d = np.array(bias_data)
clipping = sigma_clip(bias_3d, cenfunc='median', sigma=3, axis=0)
      
median_bias = np.ma.median(clipping, axis=0)

data_for_plot = median_bias.filled(np.nan)
vmin = np.nanpercentile(data_for_plot, 5)
vmax = np.nanpercentile(data_for_plot, 95)

plt.imshow(data_for_plot, origin='lower', cmap = plt.cm.plasma, vmin=vmin, vmax=vmax)
plt.savefig(Path("Eagle_Bias.png"), dpi=150)
plt.show()
    
    # Save FITS file
output_path = Path("Eagle Images") / output_file
primary = fits.PrimaryHDU(data=data_for_plot, header=fits.Header())
hdul = fits.HDUList([primary])
hdul.writeto(output_path, overwrite=True)

NameError: name 'Path' is not defined

In [ ]:
bias_filename= "work/480-arcsatpaper/eagle_bias.fits"
bias_frame = fits.getdata("eagle_bias.fits").astype('f4')

    #Creating an empty list to store dark frames
dark_bias_data = []
    
    #Calling the data from dark_list
for file in sorted(input_dir.glob("Dark*.fits")):
        dark_data = fits.getdata(file).astype('f4')
    #getting raw data and changing into float 32, then trimming so it doesnt crash
       # dark_data = dark[0].data.astype('f4')[1000:1500, 2000:2500]
        
        #Getting the exposure time
        exptime = 300
        
        #SUbtracting the bias frame from the dark frame (bias was trimmed to same numbers in other one)
        dark_NObias = dark_data - bias_frame
        
        #divinding the bias by exp time, becomes 2d array and is appended into dark_bias_data list
        dark_bias_data.append(dark_NObias / exptime)

#####
dark_3d = np.array(dark_bias_data)
clipping = sigma_clip(dark_3d, cenfunc='median', sigma=3, axis=0)
      
median_dark = np.ma.median(clipping, axis=0)

data_for_plot = median_dark.filled(np.nan)
vmin = np.nanpercentile(data_for_plot, 5)
vmax = np.nanpercentile(data_for_plot, 95)

plt.imshow(data_for_plot, origin='lower', cmap = plt.cm.plasma, vmin=vmin, vmax=vmax)
plt.savefig(Path("Eagle_Dark.png"), dpi=150)
plt.show()

    # Save FITS file
output_file= "eagle_dark.fits"
output_path = Path("Eagle Images") / output_file
primary = fits.PrimaryHDU(data=data_for_plot, header=fits.Header())
hdul = fits.HDUList([primary])
hdul.writeto(output_path, overwrite=True)

In [ ]:
#Getting data
bias_frame = fits.getdata("eagle_bias.fits").astype('f4')

    #Creating an empty list
flat_data = []

    #Opening flat list
for file in sorted(input_dir.glob("domeflat*.fits")):
    data = fits.getdata(file).astype('f4')

            #Subtracting the data by the bias
    sub_bias = data - bias_frame

            #Putting it back in my list
    flat_data.append(sub_bias)

    #Turning into array
array = np.array(flat_data)

    #clipping it with sigma 3
clipping = sigma_clip(array, cenfunc='median', sigma=3, axis=0)

    #Taking median
me_flat = np.ma.median(clipping, axis=0)

    #Normalizing the flats
median_flat = me_flat / np.ma.median(me_flat)

    #Saving everything
data_for_plot = median_flat.filled(np.nan)
vmin = np.nanpercentile(data_for_plot, 5)
vmax = np.nanpercentile(data_for_plot, 95)

plt.imshow(data_for_plot, origin='lower', cmap = plt.cm.plasma, vmin=vmin, vmax=vmax)
plt.savefig(Path("Eagle_Flat.png"), dpi=150)
plt.show()

    # Save FITS file
output_file= "eagle_flat.fits"
output_path = Path("Eagle Images") / output_file
primary = fits.PrimaryHDU(data=data_for_plot, header=fits.Header())
hdul = fits.HDUList([primary])
hdul.writeto(output_path, overwrite=True)

In [ ]:
bias_frame = fits.getdata("eagle_bias.fits").astype('f4')
eagle_im = fits.getdata("eagle_bias.fits").astype('f4')
    #Creating an empty list to store dark frames
dark_bias_data = []
    
    #Calling the data from dark_list
for file in sorted(input_dir.glob("Dark*.fits")):
        dark_data = fits.getdata(file).astype('f4')

with fits.open(eagle_im) as im:
    image_data = im[0].data.astype('f4')

    #Creating an empty list to store data in
rows = []

    #Doing a loop over the star
for p in positions:
        #Creating dictionary with stars position
    row = {'xcenter': p[0], 'ycenter': p[1]}

        #Looping over the radii for each star
    for r in radii:
            
            # Doing the aperture and annulus stuff for one star
        aperture = CircularAperture(p, r=r)
        annulus = CircularAnnulus(p, r_in=sky_radius_in, r_out=sky_radius_in + sky_annulus_width)

            # Calculating the aperture and annulus photometry
        phot_ap = aperture_photometry(image_data, aperture)
        stats = ApertureStats(image_data, annulus, sigma_clip=None)

            # finding area of circle and subtracting the sky background to find total flux
        aperture_area = aperture.area
        background = stats.median * aperture_area
        flux = phot_ap['aperture_sum'][0] - background

            #Storing results in dictionary with star position and flux 
        row[f'flux_r{int(r)}'] = flux

        #Putting the data into emtpy list
    rows.append(row)

    # Creating astropy table from dictionary data and adding radii/sky as metadata stuff
table = Table(rows)
table.meta['radii'] = radii
table.meta['sky_radius'] = sky_radius_in

##########
output_filename="radial_profile.png"
    #Calling radii from aperture data as a array        
radii = np.array(aperture_photometry_data.meta['radii'], dtype = float)

    #Getting the sky radius, also from aperture data
sky_radius = aperture_photometry_data.meta['sky_radius']

    #taking first flux data for each r 
fluxes = [aperture_photometry_data[f'flux_r{int(r)}'][0] for r in radii]

    #Plotting radii and fluxes
plt.figure()
plt.plot(radii, fluxes, marker='o', label='Target')

    #Sky radius position on graph 
plt.axvline(sky_radius, color='gray', linestyle='--', label='Sky radius')

    #Making it look nice
plt.xlabel('Radius')
plt.ylabel('Flux')
plt.legend()
plt.tight_layout()

    #Saving everything
plt.savefig(output_filename)
plt.close()

In [ ]:
#Opening each file and turning it into float 32
bias_frame = fits.getdata("eagle_bias.fits").astype('f4')
flat_frame = fits.getdata("eagle_flat.fits").astype('f4')
dark_frame = fits.getdata("eagle_dark.fits").astype('f4')

sci_data = fits.getdata("EAGLE_H-Alpha_20250603_102159.fits").astype('f4')

        #Getting the exposure time from header
exptime = 600

array = np.array(sci_data)

    #clipping it with sigma 3
sci_clip = sigma_clip(array, cenfunc='median', sigma=3, axis=0)
  
    #Removing bias signal
sub_bias = sci_clip - bias_frame

    #Subtracting dark frame with corrected exptime
dark_corrected = sub_bias - dark_frame * exptime

    #Normalizing the flat
flat_norm = flat_frame / np.mean(flat_frame)

    #Normalzing even more 
final_sci = dark_corrected / flat_norm
    
    #Getting rid of cosmic rays
#mask, cleaned = detect_cosmics(corrected_science)
#reduced_science = cleaned

    #Saving
data_for_plot = final_sci.filled(np.nan)
vmin = np.nanpercentile(data_for_plot, 5)
vmax = np.nanpercentile(data_for_plot, 95)

plt.imshow(data_for_plot, origin='lower', cmap = plt.cm.plasma, vmin=vmin, vmax=vmax)
plt.savefig(Path("Eagle_Final.png"), dpi=150)
plt.show()

    # Save FITS file
output_file= "eagle_science.fits"
output_path = Path("Eagle Images") / output_file
primary = fits.PrimaryHDU(data=data_for_plot, header=fits.Header())
hdul = fits.HDUList([primary])
hdul.writeto(output_path, overwrite=True)